# Import Modules

## Standard Packages

In [1]:
import os
import sys
import os.path as path
import psutil
import glob
import random
import numpy as np
import pandas as pd
import xarray as xr
import pickle
import json
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
plt.style.use('seaborn-white')
from datetime import date, datetime, timedelta, time
from timeit import default_timer as timer

/tmp/ipykernel_1173302/1258788010.py:14: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


## User-Defined Functions

In [2]:
from Extract_DFM_Reanalysis_Data_Helper import *

# Global Start Time and Memory

In [3]:
global_start_time = timer()
process = psutil.Process(os.getpid())
global_initial_memory = process.memory_info().rss

# Read the Input JSON File

### Input file name when using jupyter notebook

In [4]:
input_json_file = '/p/lustre2/jha3/Wildfire/Wildfire_LDRD_SI/01_WRF_Nelson_Data_Extracted/InputJsonFiles/input_json_extract_data_00.json'

### Input file name when using python script on command line

In [5]:
#input_json_file = sys.argv[0]

In [6]:
input_json_file

'/p/lustre2/jha3/Wildfire/Wildfire_LDRD_SI/01_WRF_Nelson_Data_Extracted/InputJsonFiles/input_json_extract_data_00.json'

### Load the Input JSON File

In [7]:
with open(input_json_file) as input_json_file_handle:
    input_json_data = json.load(input_json_file_handle)

In [8]:
input_json_data

{'paths': {'data_files_location': '/p/vast1/climres/DFM_reanalysis',
  'extracted_data_base_loc': '/p/lustre2/jha3/Wildfire/Wildfire_LDRD_SI/01_WRF_Nelson_Data_Extracted'},
 'data_set_defn': {'data_set_count': 0,
  'percent_files_to_use': 0.00817755098,
  'percent_grid_points_to_use': 0.0063656565,
  'max_history_to_consider': 8,
  'history_interval': 2},
 'nevada_data': {'remove_nevada': True,
  'j_nevada': 80,
  'i_nevada': 250,
  'j_anchor': 310,
  'i_anchor': 250},
 'fire_flags': {'remove_fire_data_from_train_test': False,
  'extract_fire_data': True},
 'fire_time_stamps': {'Woosley': {'Ref': '2018-11-08_22',
   'Start': '2018-11-01_00',
   'End': '2018-11-15_00'},
  'Creek': {'Ref': '2020-09-04_00',
   'Start': '2020-08-28_00',
   'End': '2020-09-11_00'}},
 'clip_data': {'x_clip': [100, 301], 'y_clip': [100, 301]}}

# Variables to be Used for Extracting WRF Data

## DataSet Defintion

In [ ]:
# The current data set params
data_set_count = 0

percent_files_to_use = 0.00817755098  # f1 = what percent of available files to use
percent_grid_points_to_use = 2*0.00315656565  # f2 = what percent of grid points to use
max_history_to_consider = 8 # n_history in hours
history_interval        = 2

## Flags, Other Params etc.

### Nevada Data

In [ ]:
remove_nevada = True
j_nevada, i_nevada = 80, 250
j_anchor, i_anchor = 310, 250

### Remove Fire Data

In [ ]:
# Test fort all the 4 combinations
remove_fire_data_from_train_test = False
extract_fire_data = True

### Clip Data

In [ ]:
x_clip = [100, 301]
y_clip = [100, 301]

## Paths and File Names

#### Global

In [ ]:
# WRF data set location and the extracted data set location
data_files_location = '/p/vast1/climres/DFM_reanalysis'
extracted_data_base_loc = '/p/lustre2/jha3/Wildfire/Wildfire_LDRD_SI/SJSU/01_WRF_Nelson_Data_Extracted'

#### DataSet Specific (Train and Test)

In [ ]:
data_set_name = 'data_train_test_extracted_%02d'%(data_set_count)

extracted_data_loc = os.path.join(extracted_data_base_loc, data_set_name)
os.system('mkdir -p %s'%extracted_data_loc)

collection_of_read_data_files = '{}_files_read.pkl'.format(data_set_name)
extracted_data_file_name = '{}_df.pkl'.format(data_set_name)

tab_data_file_name = '{}_tab_data.csv'.format(data_set_name)

#### DataSet Specific (Fire)

In [ ]:
fire_data_set_name = 'data_fire_extracted_%02d'%(data_set_count)

fire_data_loc = os.path.join(extracted_data_base_loc, fire_data_set_name)
os.system('mkdir -p %s'%fire_data_loc)

fire_data_file_name = '{}.pkl'.format(fire_data_set_name)

## Relevant Fire TimeStamps

In [ ]:
fire_time_stamps = {
    'Woosley': {'Ref': '2018-11-08_22', 'Start': '2018-11-01_00', 'End': '2018-11-15_00'},
    'Creek': {'Ref': '2020-09-04_00', 'Start': '2020-08-28_00', 'End': '2020-09-11_00'}
}

# Generate seed for the random number generator

In [ ]:
seed = generate_seed()
random_state = init_random_generator(seed)

# File Names

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
data_files_list_all = get_data_file_names(data_files_location)
data_files_list = data_files_list_all
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "get_data_file_names" computing time: {:.3f} s'.format(module_end_time - module_start_time))

# Remove Files Corresponding To Fire Data

## Get Indices for Fire Time Stamps

In [ ]:
if remove_fire_data_from_train_test or extract_fire_data:
    module_start_time = timer()
    module_initial_memory = process.memory_info().rss
    fire_time_indices = get_fire_time_indices (fire_time_stamps, data_files_list_all)
    module_final_memory = process.memory_info().rss
    module_end_time = timer()
    module_memory_consumed = module_final_memory - module_initial_memory
    print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
    print('Module "get_fire_time_indices" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Remove the Files with Indices for Fire Time Stamps

In [ ]:
if remove_fire_data_from_train_test:
    module_start_time = timer()
    module_initial_memory = process.memory_info().rss
    data_files_list = remove_data_around_fire (fire_time_indices, data_files_list)
    module_final_memory = process.memory_info().rss
    module_end_time = timer()
    module_memory_consumed = module_final_memory - module_initial_memory
    print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
    print('Module "remove_data_around_fire" computing time: {:.3f} s'.format(module_end_time - module_start_time))

In [ ]:
#len(data_files_list)

# Deal with just first 100 files to check for correctness of script. Be sure to undo this

In [ ]:
#data_files_list = data_files_list[0:30]

# Downsample Files

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
sampled_file_indices, sampled_data_files = downsample_data_files (data_files_list, percent_files_to_use, max_history_to_consider, random_state)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "downsample_data_files" computing time: {:.3f} s'.format(module_end_time - module_start_time))

# Get History File Indices

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
history_file_indices = get_history_file_indices (sampled_file_indices, max_history_to_consider, history_interval)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "get_history_file_indices" computing time: {:.3f} s'.format(module_end_time - module_start_time))

# Create timestamps and datetime of downsampled data files

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
sampled_time_stamps, sampled_datetime = get_datetime_for_data_files (sampled_data_files)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "get_datetime_for_data_files" computing time: {:.3f} s'.format(module_end_time - module_start_time))

# Create DataFrame using sampled file indices, filenames, timestamps, and datetime

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
df_sampled_time = create_df_sampled_time (sampled_file_indices, sampled_data_files, sampled_time_stamps, sampled_datetime, history_file_indices)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "create_df_sampled_time" computing time: {:.3f} s'.format(module_end_time - module_start_time))

In [ ]:
#df_sampled_time[df_sampled_time['ref_time_indices'] < max_history_to_consider+10]

In [ ]:
df_sampled_time.head(30)

# Plot Sampled Datetime

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
plot_sampled_datetime (df_sampled_time, extracted_data_loc)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "plot_sampled_datetime" computing time: {:.3f} s'.format(module_end_time - module_start_time))

# Read Quantities in a Selected Data File

## Read the Data in a Specified or Randomly Selected File

In [ ]:
prescribe_file = True
if prescribe_file:
    data_file_to_read = 'wrf_2018-11-07_16.nc'
    timestamp_to_read = data_file_to_read.split('_')[1] + '_' + \
                        data_file_to_read.split('_')[2].split('.')[0]
else:
    random_ind_of_downsampled_files = random.choice(range(len(sampled_file_indices)))
    file_ind_to_read = sampled_file_indices[random_ind_of_downsampled_files]
    data_file_to_read = sampled_data_files[random_ind_of_downsampled_files]
    timestamp_to_read = sampled_time_stamps[random_ind_of_downsampled_files]

In [ ]:
data_file_to_read, timestamp_to_read

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
data_at_timestamp = read_single_data_file (data_files_location, data_file_to_read, timestamp_to_read)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "read_single_data_file" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Processing Elevation Data into Pos, Neg, and Zero

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
data_at_timestamp = process_elevation_at_timestamp (data_at_timestamp)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "process_elevation_at_timestamp" computing time: {:.3f} s'.format(module_end_time - module_start_time))

# Get and Plot Grid Indices (All and Considerable)

## Get Grid Indices

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
grid_indices_all, grid_indices_valid, grid_indices_all_flat, grid_indices_valid_flat = \
                        get_grid_indices_all (data_files_location, sampled_file_indices, \
                                              sampled_data_files, sampled_time_stamps, \
                                              x_clip, y_clip, \
                                              j_nevada, i_nevada, j_anchor, i_anchor, remove_nevada)

module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "get_grid_indices_all" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Reconstruct Grid Indices

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
grid_indices_valid_reconst, grid_indices_valid_bool, valid_grid_ind_to_coord = \
                reconstruct_valid_grid_indices (grid_indices_valid_flat, data_at_timestamp)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "reconstruct_valid_grid_indices" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Plot Grid Indices

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
plot_contours_of_indices (data_at_timestamp, grid_indices_all, grid_indices_valid, \
                          grid_indices_valid_bool, grid_indices_valid_reconst, \
                          extracted_data_loc)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "plot_contours_of_indices" computing time: {:.3f} s'.format(module_end_time - module_start_time))

In [ ]:
#len(grid_indices_valid_flat)

# Plot Quantities in the Selected Data File

## Plot the Contours of QoIs for the Data Just Read Above

In [ ]:
qoi_to_plot = ['HGT', 'HGT_UPD', 'U10', 'V10', 'FMC_1hr', 'FMC_10hr', 'FMC_100hr', 'T2', 'RH', 'PRECIP', 'PSFC', 'SWDOWN']

### Unmasked Data

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
plot_contours_at_timestamp (data_at_timestamp, qoi_to_plot, extracted_data_loc, grid_indices_valid, masked = False)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "plot_contours_at_timestamp" computing time: {:.3f} s'.format(module_end_time - module_start_time))

### Masked Data

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
plot_contours_at_timestamp (data_at_timestamp, qoi_to_plot, extracted_data_loc, grid_indices_valid, masked = True)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "plot_contours_at_timestamp" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Plot the PDFs of QoIs for the Data Just Read Above

In [ ]:
qoi_to_plot = ['HGT', 'Q2', 'U10', 'V10', 'FMC_1hr', 'FMC_10hr', 'FMC_100hr', 'T2', 'RH', 'PRECIP', 'PSFC', 'SWDOWN']

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
plot_pdf_at_timestamp (data_at_timestamp, qoi_to_plot, extracted_data_loc)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "plot_pdf_at_timestamp" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Plot the Contours of QoIs With Colorbars

In [ ]:
qoi_to_plot = qoi_to_plot = ['FMC_1hr', 'FMC_10hr', 'FMC_100hr']
cont_levels_count = 31

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
plot_contours_at_timestamp2 (data_at_timestamp, timestamp_to_read, qoi_to_plot, extracted_data_loc, grid_indices_valid, cont_levels_count, masked = True, qoi_cont_range = [0, 0.3])
#plot_contours_at_timestamp2 (data_at_timestamp, timestamp_to_read, qoi_to_plot, extracted_data_loc, grid_indices_valid, cont_levels_count, masked = True)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "plot_contours_at_timestamp2" computing time: {:.3f} s'.format(module_end_time - module_start_time))

# Sample and Plot Grid Indices for Each Sampled Ref Time

## Sample Grid Indices

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
grid_indices_selected, j_indices_selected, i_indices_selected = \
    sample_grid_indices (sampled_file_indices, percent_grid_points_to_use, \
                         grid_indices_valid_flat, valid_grid_ind_to_coord)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "sample_grid_indices" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Plot Sampled Grid Indices

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
plot_sampled_grid_points (grid_indices_selected, extracted_data_loc)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "plot_sampled_grid_points" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Plot Sampled Grid Indices in 3D

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
plot_sampled_grid_points_3D (j_indices_selected, i_indices_selected, extracted_data_loc, (6, 6))
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "plot_sampled_grid_points_3D" computing time: {:.3f} s'.format(module_end_time - module_start_time))

# Create a Dict of Time Indices and Grid Indices

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
time_grid_indices_list_dict, time_grid_indices_list_count, time_grid_indices_set_dict, time_grid_indices_set_count = \
    create_time_grid_indices_map (sampled_file_indices, history_file_indices, grid_indices_selected)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "create_time_grid_indices_map" computing time: {:.3f} s'.format(module_end_time - module_start_time))

In [ ]:
#len(time_grid_indices_list_dict.keys())

In [ ]:
#len(time_grid_indices_set_dict.keys())

In [ ]:
#time_grid_indices_list_dict
#time_grid_indices_list_count
#time_grid_indices_set_dict
#time_grid_indices_set_count

# Read Files At All Possible Time Indices (Ref + History)

## Read Data at All Times

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss

file_indices_to_read = list(time_grid_indices_list_dict.keys())
data_files_to_read, time_stamps_to_read, file_indices_data_dict = \
                read_data_all_possible_times (file_indices_to_read, data_files_list, \
                                             data_files_location)

module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "read_data_all_possible_times" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Save Data In a Pickle File

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
save_data_read_at_all_possible_times (file_indices_to_read, data_files_to_read, \
                                      time_stamps_to_read, file_indices_data_dict, \
                                      extracted_data_loc, collection_of_read_data_files)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "save_data_read_at_all_possible_times" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Delete Data No Longer Needed To Free Memory

In [ ]:
global_final_memory = process.memory_info().rss
global_memory_consumed = global_final_memory - global_initial_memory
print('Total memory consumed so far: {:.3f} MB'.format(global_memory_consumed/(1024*1024)))

print('Deleting Some Variables')
del file_indices_data_dict

global_final_memory = process.memory_info().rss
global_memory_consumed = global_final_memory - global_initial_memory
print('Total memory consumed so far: {:.3f} MB'.format(global_memory_consumed/(1024*1024)))

## Read Data at All Possible Times Saved in a Pickle File

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
collection_of_read_data = read_data_from_pickle_all_possible_times (extracted_data_loc, \
                                                              collection_of_read_data_files)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "read_data_from_pickle_all_possible_times" computing time: {:.3f} s'.format(module_end_time - module_start_time))

In [ ]:
#collection_of_read_data['file_indices_data_dict'].keys()

## Plot Data in a Few Files

In [ ]:
qoi_to_plot = ['HGT', 'HGT_UPD', 'U10', 'V10', 'FMC_1hr', 'FMC_10hr', 'FMC_100hr', 'T2', 'RH', 'PRECIP', 'PSFC', 'SWDOWN']

In [ ]:
'''
num_files = 1
file_indices_data_dict = collection_of_read_data['file_indices_data_dict']
file_indices_to_read = list(file_indices_data_dict.keys())
for file_index_to_read in file_indices_to_read[0:num_files]:
    data_at_timestamp = file_indices_data_dict[file_index_to_read]
    plot_contours_at_timestamp (data_at_timestamp, qoi_to_plot, extracted_data_loc, \
                                grid_indices_valid, masked = True)
'''

# Extract and Save Data at The Sampled Time and Grid Points

## Extract Data at The Sampled Time and Grid Points

In [ ]:
data_at_times = collection_of_read_data['file_indices_data_dict']
#data_at_times.keys()

In [ ]:
module_start_time = timer()
module_initial_memory = process.memory_info().rss
df = create_dataframe_FM_atm_data (data_at_times, \
                                   sampled_file_indices, history_file_indices, \
                                   sampled_time_stamps, history_interval, \
                                   grid_indices_selected, \
                                   j_indices_selected, i_indices_selected)
module_final_memory = process.memory_info().rss
module_end_time = timer()
module_memory_consumed = module_final_memory - module_initial_memory
print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
print('Module "create_dataframe_FM_atm_data" computing time: {:.3f} s'.format(module_end_time - module_start_time))

In [ ]:
#df.head(5)

## Save The Data Extracted  at Sampled Time and Grid Points

In [ ]:
df.to_pickle(os.path.join(extracted_data_loc, extracted_data_file_name))

## Load and Test The Extracted Data Saved in Pickle File

In [ ]:
df_from_pickle = pd.read_pickle(os.path.join(extracted_data_loc, extracted_data_file_name))
df_from_pickle.head(5)

# Save Other Relevant Info in A CSV File

In [ ]:
data_for_csv = { 'max_history_to_consider':    [max_history_to_consider],
                 'history_interval':           [history_interval],
                 'num_hist_indices':           [len(history_file_indices[0])],
                 'num_total_files':            [len(data_files_list)],
                 'percent_files_to_use':       [percent_files_to_use],
                 'num_sampled_times':          [grid_indices_selected.shape[0]],
                 'num_data_files_to_read':     [len(data_files_to_read)],
                 'num_grid_points_sn':         [grid_indices_all.shape[0]],
                 'num_grid_points_we':         [grid_indices_all.shape[1]],
                 'num_total_grid_points':      [len(grid_indices_all_flat)],
                 'num_valid_grid_points':      [len(grid_indices_valid_flat)],
                 'percent_grid_points_to_use': [percent_grid_points_to_use],
                 'num_sampled_grid_points':    [grid_indices_selected.shape[1]],
                 'num_data_points':            [len(df)]               
}
tabulated_data = pd.DataFrame(data_for_csv)
tabulated_data.to_csv(os.path.join(extracted_data_loc, tab_data_file_name), index = False)

In [ ]:
tabulated_data

# Extract Fire Data

## Read Fire Data

In [ ]:
if extract_fire_data:
    module_start_time = timer()
    module_initial_memory = process.memory_info().rss

    fire_time_indices, fire_data = read_fire_data (\
                                fire_time_indices, max_history_to_consider, history_interval, \
                                data_files_list_all, data_files_location)

    module_final_memory = process.memory_info().rss
    module_end_time = timer()
    module_memory_consumed = module_final_memory - module_initial_memory
    print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
    print('Module "read_fire_data" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Create DataFrame for Fire Data

In [ ]:
if extract_fire_data:
    module_start_time = timer()
    module_initial_memory = process.memory_info().rss
    fire_data_extracted = dict()

    for fire_name in fire_time_indices.keys():
        data_at_times = fire_data[fire_name]['fire_file_indices_data_dict']
        df_fire = create_dataframe_FM_atm_data_fire (fire_name, fire_time_indices, data_at_times, \
                                                     history_interval, \
                                                     grid_indices_valid_flat, valid_grid_ind_to_coord)
        fire_data_extracted[fire_name] = df_fire

    module_final_memory = process.memory_info().rss
    module_end_time = timer()
    module_memory_consumed = module_final_memory - module_initial_memory
    print('Module memory consumed: {:.3f} MB'.format(module_memory_consumed/(1024*1024)))
    print('Module "create_dataframe_FM_atm_data_fire" computing time: {:.3f} s'.format(module_end_time - module_start_time))

## Save DataFrame for Fire Data

In [ ]:
if extract_fire_data:
    fire_data_file_handle = open(os.path.join(fire_data_loc, fire_data_file_name), 'wb')
    pickle.dump(fire_data_extracted, fire_data_file_handle)
    fire_data_file_handle.close()
    print('Wrote fire data in "{}" at "{}"'.format(fire_data_file_name, fire_data_loc))

## Load and Test The Extracted Fire Data Saved in Pickle File

In [ ]:
if extract_fire_data:
    fire_data_file_handle = open(os.path.join(fire_data_loc, fire_data_file_name), 'rb')
    fire_data_pickled = pickle.load(fire_data_file_handle)
    fire_data_file_handle.close()
    print('Read fire data from "{}" at "{}"'.format(fire_data_file_name, fire_data_loc))

In [ ]:
if extract_fire_data:
    fire_data_pickled['Woosley'].head(5)

## Delete Fire Data No Longer Needed

In [ ]:
if extract_fire_data:
    del fire_data, data_at_times, fire_data_extracted

# Global End Time and Memory

In [ ]:
global_final_memory = process.memory_info().rss
global_end_time = timer()
global_memory_consumed = global_final_memory - global_initial_memory
print('Total memory consumed: {:.3f} MB'.format(global_memory_consumed/(1024*1024)))
print('Total computing time: {:.3f} s'.format(global_end_time - global_start_time))
print('=========================================================================')
print("SUCCESS: Done Extraction of Data")